In [49]:
%matplotlib tk

import matplotlib.pyplot as plt
import sys
import os
import numpy as np
import pandas as pd
import glob as glob
import time
from tqdm import tqdm
import pickle
from omegaconf import OmegaConf

def read_pickle(file, verbose=False):
    if verbose: print('loading from..', file)
    with open(file, 'rb') as f:
        return pickle.load(f)


# add custom paths
sys.path.extend([os.path.abspath('../../../assetto_corsa_gym')])
import AssettoCorsaEnv.assettoCorsa as assettoCorsa

# Configure the logging system
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.INFO,  # Set the logging level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  # Format of the log messages
    datefmt='%Y-%m-%d %H:%M:%S',  # Format of the timestamp
)


In [50]:
sys.path.append(os.path.abspath(os.path.join('../..')))
import AssettoCorsaEnv.assettoCorsa as assettoCorsa
from AssettoCorsaEnv.curvature import curvature_splines

In [51]:
config = OmegaConf.load("../../../config.yml")
client = assettoCorsa.make_client_only(config.AssettoCorsa)

Failed to open vJoy device: [Errno 2] No such file or directory: '/dev/input/event16'


In [52]:
tracks_path = "."

# Get track's bounds and racing line from AC

In [53]:
#track_file_name, track_file, ref_line_file = env.export_track_and_racing_line(output_path=tracks_path)

track_file_name, track_file, ref_line_file, static_info = client.export_track_and_racing_line(output_path=tracks_path)

INFO:AssettoCorsaEnv.ac_client:Saving track to ./ks_zandvoort.csv
INFO:AssettoCorsaEnv.ac_client:Saving racing line ./ks_zandvoort-racing_line.csv


{'LastSplits': [0, 0], 'CAR_WHEEL_R': [0.2750000059604645, 0.2750000059604645, 0.2849999964237213, 0.2849999964237213], 'isCarInPitlane': 0, 'isCarInPit': 0, 'penaltiesEnabled': 0, 'CarName': 'tatuusfa1', 'TrackName': 'ks_zandvoort', 'TrackConfiguration': '', 'WindDirection': 359, 'CAR_WHEELBASE': 2.6496553896655883, 'WindSpeed': 0, 'autoShifterOn': 0, 'CAR_TRACK': 1.4594446404100356, 'TrackLength': 4189.69091796875, 'TrackFullName': 'ks_zandvoort'}


In [54]:
print(f'TrackFullName: {static_info["TrackFullName"]}  TrackLength: {static_info["TrackLength"]}   TrackName: {static_info["TrackName"]}  TrackConfiguration: {static_info["TrackConfiguration"]}')
static_info

TrackFullName: ks_zandvoort  TrackLength: 4189.69091796875   TrackName: ks_zandvoort  TrackConfiguration: 


{'LastSplits': [0, 0],
 'CAR_WHEEL_R': [0.2750000059604645,
  0.2750000059604645,
  0.2849999964237213,
  0.2849999964237213],
 'isCarInPitlane': 0,
 'isCarInPit': 0,
 'penaltiesEnabled': 0,
 'CarName': 'tatuusfa1',
 'TrackName': 'ks_zandvoort',
 'TrackConfiguration': '',
 'WindDirection': 359,
 'CAR_WHEELBASE': 2.6496553896655883,
 'WindSpeed': 0,
 'autoShifterOn': 0,
 'CAR_TRACK': 1.4594446404100356,
 'TrackLength': 4189.69091796875,
 'TrackFullName': 'ks_zandvoort'}

# Check racing line

In [55]:
df = pd.read_csv(ref_line_file)
file_channels = ["pos_x", "pos_y"]

ts = df[file_channels].values

In [56]:
#plt.scatter(ts[:,0],  ts[:,1], s=0.1, color="gray")
#plt.scatter(ts[0,0],  ts[0,1], s=0.5, color="red")

In [57]:
track_file_name

'ks_zandvoort'

# check

- adapt this to the track
- curvature should be close to zero at the end and beginning of the track (main straight)

In [58]:

if track_file_name == "ks_red_bull_ring-layout_gp":
    curvature = curvature_splines( ts[:,0],  ts[:,1] , error=0.1)
elif track_file_name == "indianapolis_sp":
    curvature = curvature_splines( ts[:,0],  ts[:,1] , error=.25)
elif track_file_name == "imola":
    curvature = curvature_splines( ts[:,0],  ts[:,1] , error=1)
else:
    curvature = curvature_splines( ts[:,0],  ts[:,1] , error=1.)
    curvature[2934:] = 0. # force zero in the last part

curvature = curvature.reshape(-1,1)
plt.plot(curvature)

# Create occupancy grip map

In [59]:
track_grid_file = tracks_path + os.sep + f"{track_file_name}.pkl"
track_grid_file

'.\\ks_zandvoort.pkl'

In [60]:
from AssettoCorsaEnv.track import Track, in_quadrilateral

track = Track(track_file)

INFO:AssettoCorsaEnv.track:Track loaded from: ./ks_zandvoort.csv Found 2710 segments downsampled by 10= 271 segments


In [61]:
plt.plot(track.left_border_x, track.left_border_y)
plt.plot(track.right_border_x, track.right_border_y)
plt.scatter(track.right_border_x, track.right_border_y, color='green', s=0.1)
plt.scatter(track.left_border_x[0], track.left_border_y[0], color='red')

# racing line
plt.scatter(ts[:,0],  ts[:,1], s=0.1, color="gray")
plt.scatter(ts[0,0],  ts[0,1], s=0.5, color="red")

In [62]:
# add 1 m just in case
min_x = min( np.min(track.right_border_x), np.min(track.left_border_x) ) - 1
min_y = min( np.min(track.right_border_y), np.min(track.left_border_y) ) - 1

max_x = max( np.max(track.right_border_x), np.max(track.left_border_x) ) + 1
max_y = max( np.max(track.right_border_y), np.max(track.left_border_y) ) + 1

min_x, min_y, max_x, max_y


(-451.6979240852052, -571.255285030138, 397.6443523702942, 256.35558562188294)

In [63]:
##
## quick test hardcode a small grid
##
# max_x = min_x + 200
# max_y = min_y + 220
# max_x, max_y

In [64]:
cell_size = .1 # m

In [65]:
x_range = np.arange(min_x, max_x, cell_size)
y_range = np.arange(min_y, max_y, cell_size)

x_range.shape, y_range.shape


((8494,), (8277,))

In [66]:
xx, yy = np.meshgrid(x_range, y_range)
xx.shape, yy.shape


((8277, 8494), (8277, 8494))

In [67]:
points = np.vstack([xx.ravel(), yy.ravel()]).T  


In [68]:
track.left_border_y.shape

(271,)

In [69]:
show_segments = -1
for n in range(len(track.right_border_x[:show_segments]))[::]:
    plt.plot([track.left_border_x[n], track.right_border_x[n]],
             [track.left_border_y[n], track.right_border_y[n]], color='blue')

In [70]:
n_segments = track.num_segments - 1
n_segments

270

In [71]:
found_base = np.zeros(points.shape[0], dtype='bool')

def calc_points_inside_quad():
    return found

for segment in tqdm(range(n_segments)):
    corners = np.array( [track.lr_track[0+segment*2:4+segment*2]] )
    found = in_quadrilateral(corners, points)
    found_base = found_base + found

# Add
corners = np.concatenate([ [track.lr_track[-2:]],  [track.lr_track[:2]]] , axis=1)
found = in_quadrilateral(corners, points)
found_base = found_base + found

found_base = found_base.astype('ubyte')

100%|██████████| 270/270 [11:39<00:00,  2.59s/it]


In [72]:
found_base = found_base.astype('ubyte')

In [73]:
# # plot the track
plt.plot(track.right_border_x, track.right_border_y)
plt.plot(track.left_border_x, track.left_border_y)
#plt.plot(track.middle_x, track.middle_y)

#plt.scatter(corners[0,:,0], corners[0,:,1],s=0.02)
#plt.scatter(points[:,0], points[:,1], s=0.01)
plt.scatter(points[found_base == 1][:,0], points[found_base == 1][:,1], s=0.02)
plt.axis("equal")

(-607.9328202894656, 2851.23489620426, -611.5358285627391, 296.636129154484)

# export


In [74]:
cell_size

0.1

In [75]:
import pickle
track_map = {"cell_size": cell_size,
        "min_x": min_x,
        "min_y": min_y,
        "max_x": max_x,
        "max_y": max_y,
        "grid": found_base.astype('ubyte')
        }

export_file = f"{tracks_path}/{track_file_name}_{cell_size}m.pkl"

with open(export_file,'wb') as f:
    pickle.dump(track_map, f)

del track_map



print(f"saved to {export_file}")

saved to ./ks_zandvoort_0.1m.pkl
